In [1]:
import json
import os
import textwrap

import langchain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai.model_garden import ChatAnthropicVertex
from langchain_openai import ChatOpenAI

import llm_cache_stats_wrapper
import simple_llm_cache

# In order to make it easy to run work projects and personal AI experiments, override these key values with the value of *_PERSONAL, if set.
if "OPENAI_API_KEY_PERSONAL" in os.environ:
    print("Using key from OPENAI_API_KEY_PERSONAL environment variable")
    os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY_PERSONAL"]
if "ANTHROPIC_API_KEY_PERSONAL" in os.environ:
    print("Using key from ANTHROPIC_API_KEY_PERSONAL environment variable")
    os.environ["ANTHROPIC_API_KEY"] = os.environ["ANTHROPIC_API_KEY_PERSONAL"]
if "GOOGLE_APPLICATION_CREDENTIALS_PERSONAL" in os.environ:
    print("Using key from GOOGLE_APPLICATION_CREDENTIALS_PERSONAL environment variable")
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.environ["GOOGLE_APPLICATION_CREDENTIALS_PERSONAL"]

verbose = False
temperature = 0.5
vertex_model_name_prefix = "vertexai-"

langchain.llm_cache = llm_cache_stats_wrapper.LlmCacheStatsWrapper(simple_llm_cache.SimpleLlmCache("llm-cache.json"))

def dump_cache_stats_since_last_call():
    print(langchain.llm_cache.get_cache_stats_summary())
    langchain.llm_cache.clear_cache_stats()

template = """List all the countries in North America and their capital cities. After the name of each city, give it a nickname, placed inside quotation marks. For example, if New York were a capital, you could say New Yord "The Big Apple"."""

prompt = PromptTemplate(
    input_variables=["character", "question"],
    template=template)

response_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
            },
            "capital": {
                "type": "string",
            },
        },
        "required": ["country", "capital"],
    },
}

for model_name in [
    # "gpt-3.5-turbo",
    # "gpt-4",
    # "gpt-4o-mini",
    # "gpt-4o",
    # "claude-3-haiku-20240307",
    "gemini-1.5-pro-001",
    # "vertexai-claude-3-haiku@20240307",
]:
    if model_name.startswith("gpt-"):
        llm = ChatOpenAI(
            temperature=temperature,
            model_name = model_name,
            )
    elif model_name.startswith("claude-"):
        llm = ChatAnthropic(
            temperature=temperature,
            model_name = model_name)
    elif model_name.startswith("gemini-"):
        llm = ChatVertexAI(
            temperature=temperature,
            model_name=model_name,
            response_mime_type="application/json",
            response_schema=response_schema,
        )
    elif model_name.startswith(vertex_model_name_prefix):
        vertex_model_name = model_name[len(vertex_model_name_prefix) :]
        llm = ChatAnthropicVertex(
            temperature=temperature, model_name=vertex_model_name
        )

    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose)

    for trial in range(2):
        print(f"\n*** {model_name} trial {trial} ***")
        langchain.llm_cache.inner_cache.set_trial(trial)
        print(f"TRIAL {langchain.llm_cache.inner_cache.get_trial()}")
        output = chain.predict()
        print(textwrap.fill(output, width=80))
        print("\nFrom JSON:")
        json_output = json.loads(output)
        for item in json_output:
            print(f"{item['country']}: {item['capital']}")

print()
dump_cache_stats_since_last_call()

Using key from OPENAI_API_KEY_PERSONAL environment variable
Using key from GOOGLE_APPLICATION_CREDENTIALS_PERSONAL environment variable


C:\Users\forre\AppData\Local\Temp\ipykernel_30088\1443925633.py:90: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
c:\s\MinSampleTemplates\LangChain\simple_llm_cache.py:116: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  generations.append(loads(generation))



*** gemini-1.5-pro-001 trial 0 ***
TRIAL 0
[{"capital": "Ottawa \"Bytown\"", "country": "Canada"}, {"capital": "Mexico City
\"The City of Palaces\"", "country": "Mexico"}, {"capital": "Washington, D.C.
\"The District\"", "country": "United States"}]

From JSON:
Canada: Ottawa "Bytown"
Mexico: Mexico City "The City of Palaces"
United States: Washington, D.C. "The District"

*** gemini-1.5-pro-001 trial 1 ***
TRIAL 1
[{"capital": "Ottawa \"Bytown\"", "country": "Canada"}, {"capital": "Mexico City
\"The City of Palaces\"", "country": "Mexico"}, {"capital": "Washington, D.C.
\"The District\"", "country": "United States"}]

From JSON:
Canada: Ottawa "Bytown"
Mexico: Mexico City "The City of Palaces"
United States: Washington, D.C. "The District"

LLM Cache: 2 hits, 0 misses
           0 new input characters, 0 new output characters, 36 total input characters, 36 total output characters
           new (this run) API cost: $0.00, total (including previously-cached runs) API cost: $0.00

